# Import Libraries

In [35]:
import kagglehub
import pandas as pd

# Download the dataset

In [ ]:
# Download latest version
# https://www.kaggle.com/datasets/snap/amazon-fine-food-reviews
path = kagglehub.dataset_download("snap/amazon-fine-food-reviews")
print("Path to dataset files:", path)

In [ ]:
# Load the dataset
df = pd.read_csv(path + "/Reviews.csv")
print(df.head())

In [ ]:
# Print the information of the dataset
print(df.info())

# Extract Features and Target

In [ ]:
# Score is the rating of the product. This will be our target variable.
df_score = df["Score"]
df_score

In [ ]:
# Print the unique values of the target variable
df_score.unique()

In [ ]:
# Print the statistics of the target variable
score_statistics = df_score.describe()
score_statistics

In [ ]:
import matplotlib.pyplot as plt
# Plotting the bar chart for the value counts of the scores
score_counts = df_score.value_counts()
plt.figure(figsize=(10, 2))
score_counts.plot(kind='bar')
plt.title('Distribution of Scores')
plt.xlabel('Score')
plt.ylabel('Count')
plt.show()

In [ ]:
# Summary is the title of the review. This will constitute our features.
df_summary = df["Summary"]
df_summary

In [ ]:
# Importing numpy for generating random indices
import numpy as np
# Generating 10 random indices from the range of df_score length
rand_idxs = np.random.randint(0, len(df_score), size=10)
# Iterating over the random indices to print corresponding Score and Summary
for idx in rand_idxs:
    print(f"Score: {df_score.iloc[idx]} - Summary: {df_summary.iloc[idx]}")

In [45]:
# We zero out the data to free up memory
df = 0

# Preprocessing

The Preprocessing steps we will use are:
1. Lower Casing
2. Replacing URLs
3. Replacing Emojis
4. Replacing Usernames
5. Removing Non-Alphabets
6. Removing Consecutive letters
7. Removing Short Words
8. Removing Stopwords
9. Lemmatization

## Lowercase

In [46]:
def lowercase_text(text):
    # Convert text to lowercase.
    return str(text).lower()

In [47]:
# Apply lowercase function to all summaries
df_summary = df_summary.apply(lowercase_text)

In [ ]:
# Display a few examples to verify the transformation
print("After lowercase transformation:")
rand_idxs = np.random.randint(0, len(df_summary), size=10)
for idx in rand_idxs:  
    print(f"Score: {df_score.iloc[idx]} - Summary: {df_summary.iloc[idx]}")

## Replace URLs

In [49]:
import re  # Importing the regular expressions module

# Define a regex pattern to identify URLs in the text
url_pattern = r"(?:https?|ftp)://[^\s/$.?#].[^\s]*"

def replace_urls(text):
    """
    Replace URLs in the text with the token 'URL'.
    Prints before and after if a replacement occurs.
    """
    text_str = str(text)
    replaced_text = re.sub(url_pattern, 'URL', text_str)

    if replaced_text != text_str:
        print(f"Before: {text_str}")
        print(f"After:  {replaced_text}\n")

    return replaced_text

In [ ]:
# Apply URL replacement to all summaries
df_summary = df_summary.apply(replace_urls)

## Replacing Emojis

In [51]:
import re

# re.compile will compile the regex pattern into a regex object, necessary for 
# efficient pattern matching. This creates a reusable pattern object that can be
# used multiple times without recompiling the pattern each time, improving performance.
# u stands for Unicode
emoji_pattern = re.compile("["

    # Emoticons (e.g., 😀😁😂🤣😃😄😅😆)
    u"\U0001F600-\U0001F64F"  

    # Symbols & pictographs (e.g., 🔥🎉💡📦📱)
    u"\U0001F300-\U0001F5FF"  

    # Transport & map symbols (e.g., 🚗✈️🚀🚉)
    u"\U0001F680-\U0001F6FF"  

    # Flags (e.g., 🇺🇸🇬🇧🇨🇦 — these are pairs of regional indicators)
    u"\U0001F1E0-\U0001F1FF"  

    # Dingbats (e.g., ✂️✈️✉️⚽)
    u"\u2700-\u27BF"          

    # Supplemental Symbols & Pictographs (e.g., 🤖🥰🧠🦾)
    u"\U0001F900-\U0001F9FF"  

    # Symbols & Pictographs Extended-A (e.g., 🪄🪅🪨)
    u"\U0001FA70-\U0001FAFF"  

    # Miscellaneous symbols (e.g., ☀️☁️☂️⚡)
    u"\u2600-\u26FF"          

    "]+", flags=re.UNICODE)


In [52]:
# This pattern will match common text-based emoticons that aren't covered by the emoji Unicode ranges
# These emoticons are made up of regular ASCII characters like colons, parentheses, etc.
# Examples include:
# :) - happy face
# :( - sad face
# :D - laughing face
# ;) - winking face
emoticon_pattern = re.compile(r'(:\)|:\(|:D|:P|;\)|:-\)|:-D|:-P|:\'\(|:\||:\*)')

In [53]:
def remove_and_print(text):
    if emoji_pattern.search(text) or emoticon_pattern.search(text):
        print(f"Before: {text}")
        text = emoji_pattern.sub('', text)
        text = emoticon_pattern.sub('', text)
        print(f"After: {text}")
        print()
    return text

In [ ]:
df_summary = df_summary.apply(remove_and_print)

## Replacing Usernames

In [ ]:
import re

def replace_usernames(text):
    """
    Replace email addresses and true @usernames with 'USER'.
    Avoid matching embedded @ in profanity or stylized words.
    Print before and after if replacement occurs.
    """
    original = str(text)
    updated = original

    # Replace full email addresses
    updated = re.sub(r'\b[\w\.-]+@[\w\.-]+\.\w+\b', 'USER', updated)

    # Replace @usernames only when preceded by space, punctuation, or start of string
    updated = re.sub(r'(?:(?<=^)|(?<=[\s.,;!?]))@\w+\b', 'USER', updated)

    if updated != original:
        print(f"Before: {original}")
        print(f"After:  {updated}\n")
    
    return updated


# Apply username replacement to all summaries
df_summary = df_summary.apply(replace_usernames)

## Removing Non-Alphabets

In [56]:
import re

def clean_text(text, keep_punct=False):
    """
    Clean and normalize text for NLP classification tasks.
    
    Parameters:
    - text (str): The input text to be cleaned.
    - keep_punct (bool): 
        If True, retains key punctuation (. ! ?) which may carry emotional or contextual weight.
        If False, removes all non-alphabetic characters for simpler lexical analysis.
    
    Returns:
    - str: The cleaned text string, lowercased and stripped of unwanted characters.
    
    This function is designed for flexibility across different NLP tasks like sentiment analysis,
    topic classification, or spam detection. It handles:
    - Lowercasing text for normalization
    - Removing or preserving select punctuation
    - Removing digits, symbols, and special characters
    - Reducing multiple spaces to a single space
    - Optionally printing changes for debugging or logging

    When to use `keep_punct=True`:
    - Sentiment analysis: punctuation (e.g., "!", "?") can reflect strong emotion
    - Social media or informal text: expressive punctuation often carries signal
    - Sarcasm, emphasis, or tone-sensitive tasks

    When to use `keep_punct=False`:
    - Topic classification or document clustering: punctuation rarely adds value
    - Preprocessing for bag-of-words, TF-IDF, or topic modeling
    - When punctuation is inconsistent or noisy (e.g., OCR scans, scraped data)
    """
    
    # Convert input to string (safe handling)
    original = str(text)

    if keep_punct:
        # Keep only lowercase letters, spaces, and select punctuation (. ! ?)
        # Useful for capturing tone/sentiment
        cleaned = re.sub(r"[^a-z\s.!?]", "", original)
    else:
        # Keep only lowercase letters and spaces; remove all punctuation and symbols
        cleaned = re.sub(r"[^a-z\s]", "", original)

    # Normalize whitespace (collapse multiple spaces to one, strip leading/trailing)
    cleaned = re.sub(r'\s+', ' ', cleaned).strip()

    # Optional: print before and after if a change occurred
    if original != cleaned:
        print(f"Before: {text}")
        print(f"After:  {cleaned}\n")

    return cleaned

In [ ]:
# Apply non-alphabet removal to all summaries
df_summary = df_summary.apply(lambda x: clean_text(x, keep_punct=True))

## Removing Consecutive letters

In [58]:
def remove_consecutive_letters(text, max_repeat=2):
    """
    Normalize elongated words by limiting repeated characters.

    In informal or emotional text (e.g., reviews, tweets), users often repeat letters
    to add emphasis: "sooooo good", "loooove it", "greeaaat".
    
    This function reduces any character repeated more than `max_repeat` times 
    to exactly `max_repeat` occurrences (default: 2), preserving emphasis without bloating vocabulary.

    Parameters:
    - text (str): The input text
    - max_repeat (int): The maximum allowed repetitions for any character

    Returns:
    - str: Text with repeated characters normalized
    """
    text_str = str(text)
    pattern = r'(\w)\1{' + str(max_repeat) + r',}'
    cleaned = re.sub(pattern, r'\1' * max_repeat, text_str)

    # Print only if changes were made
    if cleaned != text_str:
        print(f"Before: {text_str}")
        print(f"After:  {cleaned}\n")

    return cleaned

In [ ]:
# Apply consecutive letter removal to all summaries
df_summary = df_summary.apply(lambda x: remove_consecutive_letters(x, max_repeat=2))

## Removing Short Words

In [60]:
def remove_short_words(text, min_length=3, preserve_words=None):
    """
    Remove short words from text based on a minimum length threshold.
    
    Parameters:
    - text (str): The input text
    - min_length (int): Minimum word length to keep (default = 3)
    - preserve_words (set or list): Optional set of short but important words to keep (e.g., {'no', 'not'})
    
    Returns:
    - str: Text with short words removed, except for preserved ones
    
    Notes:
    - Use with care in sentiment analysis. Important short words like 'no', 'not', 'bad' may affect meaning.
    - Best used after stopword removal or on very noisy text.
    """
    preserve = set(preserve_words or [])
    words = str(text).split()
    filtered = [word for word in words if len(word) >= min_length or word.lower() in preserve]
    result = ' '.join(filtered)

    if result != text:
        print(f"Before: {text}")
        print(f"After:  {result}\n")

    return result

In [ ]:
# Apply short word removal to all summaries
df_summary = df_summary.apply(lambda x: remove_short_words(x, min_length=3, preserve_words={'no', 'not'}))

## Removing Stopwords

In [ ]:
# NLTK (Natural Language Toolkit) is a popular library for natural language processing in Python
# https://www.nltk.org/
import nltk
from nltk.corpus import stopwords

# Download stopwords if not already downloaded
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords')

print("Sample stopwords:", list(stopwords.words('english'))[:10])

In [64]:
# Define stopwords but keep critical ones like "not"
base_stopwords = set(stopwords.words('english'))
preserve = {'no', 'not', 'nor', 'never'}
custom_stopwords = base_stopwords - preserve

In [65]:
def remove_stopwords(text):
    """
    Remove stopwords from text, preserving key negation words.

    This function uses a customized stopword list that retains important
    short words like 'not', 'no', 'nor', and 'never' which carry significant
    meaning in tasks like sentiment analysis.

    Parameters:
    - text (str): Lowercased input text

    Returns:
    - str: Text with stopwords removed, but critical negation words preserved
    """
    words = str(text).split()
    filtered = [word for word in words if word not in custom_stopwords]
    result = ' '.join(filtered)

    if result != text:
        print(f"Before: {text}")
        print(f"After:  {result}\n")

    return result

In [ ]:
# Apply remove_stopwords
df_summary = df_summary.apply(remove_stopwords)

## Lemmatization

In [ ]:
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

# Download required NLTK resources
nltk.download('wordnet')  # Download WordNet, a lexical database of English words
nltk.download('omw-1.4')  # WordNet Lemmas sometimes need this, which is a mapping of WordNet lemmas to their Part of Speech (POS) tags.
nltk.download('averaged_perceptron_tagger_eng')  # Download English POS tagger

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
# POS mapping function 
# POS tags can be: ADJ (adjective), ADV (adverb), NOUN (noun), VERB (verb), etc
def get_wordnet_pos(tag):
    # Determine the WordNet POS tag based on the first letter of the input tag
    if tag.startswith('J'):
        return wordnet.ADJ  # Adjective
    elif tag.startswith('V'):
        return wordnet.VERB  # Verb
    elif tag.startswith('N'):
        return wordnet.NOUN  # Noun
    elif tag.startswith('R'):
        return wordnet.ADV  # Adverb
    else:
        return wordnet.NOUN  # Default to Noun if no match


def lemmatize_text(text):
    """
    Lemmatize text using WordNet lemmatizer with POS tagging.

    This version prints each change along with the POS tag of the changed word.
    """
    # Convert the input text to a string to ensure compatibility
    original_text = str(text)
    # Split the text into individual words
    words = original_text.split()
    # Obtain Part of Speech (POS) tags for each word
    pos_tags = pos_tag(words)

    # Initialize lists to store lemmatized words and any changes
    lemmatized_words = []
    changes = []

    # Iterate over each word and its POS tag
    for word, tag in pos_tags:
        # Map the POS tag to a WordNet POS tag
        wn_tag = get_wordnet_pos(tag)
        # Lemmatize the word using the mapped POS tag
        lemma = lemmatizer.lemmatize(word, wn_tag)

        # Check if the lemmatized word is different from the original
        if lemma != word:
            # Record the change if a difference is found
            changes.append((word, lemma, tag))
        # Add the lemmatized word to the list
        lemmatized_words.append(lemma)

    # Join the lemmatized words back into a single string
    result = ' '.join(lemmatized_words)

    # Print only if there were changes
    if changes:
        print(f"\nOriginal: {original_text}")
        print(f"Lemmatized: {result}")
        for original, lemma, pos in changes:
            print(f"  - {original} → {lemma}  (POS: {pos})")

    return result

In [ ]:
# Apply lemmatization to all summaries
df_summary = df_summary.apply(lemmatize_text)

# Visualize

## Word Cloud for positive sentiments

In [ ]:
from wordcloud import WordCloud

# Filter summaries for df_score >= 4
filtered_summaries = df_summary[df_score >= 4]

# Combine all filtered summaries into a single string
all_summaries = " ".join(str(summary) for summary in filtered_summaries)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_summaries)

# Clear the memory
all_summaries = 0

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

## Word Cloud for negative sentiments

In [ ]:
from wordcloud import WordCloud

# Filter summaries for df_score 1
filtered_summaries = df_summary[df_score == 1]

# Combine all filtered summaries into a single string
all_summaries = " ".join(str(summary) for summary in filtered_summaries)

# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_summaries)

# Clear the memory
all_summaries = 0

# Display the word cloud
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()